# Inver-over operator - memetic alternative to classic GA

In [1]:
import random
import time
import os

In [2]:
# load distance matrix
def calculate_distance_matrix(file_name):
    lines = open(file_name).readlines()
    num_of_cities = int(lines[0].strip())
    matrix = [[0 for _ in range(num_of_cities)] for _ in range(num_of_cities)]
    
    i_line = 0
    for line in lines[1:]: # 1 line contains number of cities
        row = list(map(int, line.split())) 
        for i in range(len(row)):
            matrix[i_line][i] = row[i]
            matrix[i][i_line] = row[i]
        i_line += 1
    return matrix

# stats
def the_best_solution(population): 
    return min(population, key=lambda ind: ind[1])

def avg_solutions(population): 
    return round(sum([ind[1] for ind in population])/len(population))

# creating population
def new_individual(n): 
    individual = [i for i in range(n)]
    random.shuffle(individual)
    return individual

def initialize_population(m, n): 
    return [[new_individual(n), -1] for _ in range(m)]

# funkcje oceny osobnika w populacji
def calculate_fitness(individual, d_matrix):
    sum_dist = 0
    ind = individual[0] 
    for i in range(len(ind)-1): 
        sum_dist += d_matrix[ind[i]][ind[i+1]] # check the distance between i and (i+1) element
    
    sum_dist += d_matrix[ind[-1]][ind[0]] # add the last distance

    individual[1] = sum_dist 

def evaluate_population(population, distance_matrix):
    for individual in population:
        calculate_fitness(individual,distance_matrix)

# inver-over operator
def inver_over(num_of_iterations, population, matrix, probability):
    # for i in range(num_of_iterations):
    counter = 0
    best_solution = the_best_solution(population)[1]
    while counter != 1000:
        for individual in population:
            ind_1 = individual[0].copy()
            c_1 = random.choice(ind_1)
            ind_1_short = [i for i in ind_1 if i != c_1]
    
            while True:
                # 'mutation'
                if random.random() <= probability: 
                    c_2 = random.choice(ind_1_short)
                # 'cross-over'
                else:
                    ind_2 = random.choice(population)[0].copy()
                    try:
                        c_2 = ind_2[ind_2.index(c_1)+1]
                    except IndexError:
                        c_2 = ind_2[0]
    
                if ind_1.index(c_2) in [ind_1.index(c_1) + 1, ind_1.index(c_1) - 1]:
                    break
                else:
                    cp_1 = ind_1.index(c_1) + 1
                    cp_2 = ind_1.index(c_2) + 1
                    if cp_1 > cp_2:
                        cp_1, cp_2 = cp_2 - 1, cp_1 - 1
                    ind_1[cp_1:cp_2] = ind_1[cp_1:cp_2][::-1]
                    c_1 = c_2
                
            # fitness function
            sum_dist = 0
            for i in range(len(ind_1)-1): 
                sum_dist += matrix[ind_1[i]][ind_1[i+1]]
            
            sum_dist += matrix[ind_1[-1]][ind_1[0]]
            
            if sum_dist <= individual[1]:
                individual[0] = ind_1
                individual[1] = sum_dist
                #print('sum_dist', sum_dist, 'zastąpił', individual[1])
                
        # alternative termination condition
        counter += 1
        if the_best_solution(population)[1] < best_solution:
            best_solution = the_best_solution(population)[1]
            counter = 0

In [3]:
def read_tsp_dict(file_name):
    file_list = open(file_name).readlines()
    tsp_dict = {}
    for item in file_list:
        key, value = item.split(':')
        try:
            tsp_dict[key.strip()] = int(value.strip())
        except:
            continue
    return tsp_dict

tsp_dict = read_tsp_dict('sym_TSP_best_solutions.txt')
path_name = 'data/matrices/'
files_listed = [fname for fname in os.listdir(path_name) if os.path.isfile(os.path.join(path_name, fname))]

In [4]:
# params
pop_size = 100
num_of_iterations = 10000
probability = 0.02

for file in files_listed:
    # execution time measurement
    start_time = time.time()
    
    matrix = calculate_distance_matrix(path_name + file)
    num_of_cities = len(matrix) 
    population = initialize_population(pop_size, num_of_cities)
    
    # initial evaluation
    evaluate_population(population, matrix)

    # inver-over
    inver_over(num_of_iterations, population, matrix, probability)
    
    # end evaluation
    evaluate_population(population, matrix)
    best_solution = the_best_solution(population)
    known_opt = tsp_dict[file.split('.')[0]]
    
    print('TSP:', file.split('.')[0])
    print('Solution:', best_solution[1])
    
    if best_solution[1] == known_opt:
        print('Optimal solution achieved!')
    else:
        print('Distance to optimal solution: {} ({}%)'.format(best_solution[1] - known_opt, round(100*(best_solution[1]/known_opt)-100, 2)))
    
    time_delta = time.time() - start_time
    minutes = time_delta // 60
    seconds = time_delta - minutes * 60
    print("Execution time: {:.0f} min {:.2f} sec".format(minutes, seconds))
    print('----------')

TSP: a280
Solution: 2749
Distance to optimal solution: 170 (6.59%)
Execution time: 10 min 17.88 sec
----------
TSP: berlin52
Solution: 7542
Optimal solution achieved!
Execution time: 0 min 18.98 sec
----------
TSP: ch150
Solution: 6609
Distance to optimal solution: 81 (1.24%)
Execution time: 2 min 48.82 sec
----------
